In [8]:
## Dictionary creation for mappping -- necessary?? - yes, apparently


################################################################################


from pymongo import MongoClient
import pandas as pd

df = pd.read_csv('../temp/intermediate_mapping_file.csv')
# identifiers = ['Tumor_Sample_UUID','Matched_Norm_Sample_UUID','case_id']


client = MongoClient('localhost', 27017)
db = client.progenetixCopy ############### Copied db used here ##############
bios_coll = db[ "biosamples" ]
ind_coll = db[ "individuals" ]

ind_ids = {}

tcga_s_id = df['Tumor_Sample_UUID']
tcga_mns_id = df['Matched_Norm_Sample_UUID']
tcga_c_id = df['case_id']

idlist = df[0:20]

i = 0

for ide in tcga_c_id:
    
    q = {'info.legacy_ids': tcga_s_id[i]}
    b = bios_coll.find_one(q) ## What do they do?? ## Makes issues
    i_e_r = [{"id": "pgx:TCGA-"+ide, "label": "TCGA case_id"},] # for the individual collection??

    if ide in ind_ids:
        ind_ids[ide]["biosample_ids"].append(bios_coll.find_one({'info.legacy_ids': tcga_s_id[i]})["id"]) #combined b an q from og code
    else:
        ind_ids.update({
            ide: {
                "external_references": i_e_r,
                "biosample_ids": [tcga_s_id[i]]
            }
        })
        
    b_e_r = i_e_r.copy() # for the biosamples collection
    b_e_r.append({"id": "pgx:TCGA-"+tcga_s_id[i], "label": "TCGA sample_id"})

    for e in bios_coll.find_one({'info.legacy_ids': tcga_s_id[i]})["external_references"]:
        if not "TCGA" in e["id"]: # <= important to keep other ers
            b_e_r.append(e)
        if re.match(r'^(?:pgx:)?TCGA\-\w+?$', e["id"]):
            project = re.sub('TCGA-', '', e["id"])
            project = re.sub('pgx:', '', project)
            b_e_r.append({"id": "pgx:TCGA-"+project, "label": "TCGA "+project+" project"})

    print(ide, ind_ids[ide]["biosample_ids"])
    
    i += 1
############################################################################

becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
becedbfd-b2aa-4dde-b7f4-29e6f59ec32c
c397e6c2-ba3a-4f5e-a258-c140611192fe
c397e6c2-ba3a-4f5e-a258-c140611192fe
c397e6c2-ba3a-4f5e-a258-c140611192fe
c397e6c2-ba3a-4f5e-a258-c140611192fe
c397e6c2-ba3a-4f5e-a258-c140611192fe
c397e6c2-ba3a-4f5e-a258-c140611192fe
c397e6c2-ba3a-4f5e-a258-c140611192fe
c397e6c2-ba3a-4f5e-a258-c140611192fe
c397e6c2-ba3a-4f5e-a258-c140611192fe
c397e6c2-ba3a-4f5e-a258-c140611192fe
c397e6c2-ba3a-4f5e-a258-c140611192fe
c

In [ ]:
########## MY DICTONARY APPROACH ##############
from pymongo import MongoClient
import pandas as pd

# Load data frame from intermediate mapping file
df = pd.read_csv('../temp/intermediate_mapping_file.csv') ###### Make input later


# Prepare mongodb tools
client = MongoClient('localhost', 27017)
db = client.progenetixCopy ############### Copied db used here ##############
bios_coll = db[ "biosamples" ]
ind_coll = db[ "individuals" ]

ind_ids = {}

sample_id = df['Tumor_Sample_UUID']
mns_id = df['Matched_Norm_Sample_UUID']
case_id = df['case_id']


    case_hit = bs.find_one({"external_references.id": "pgx:TCGA-" + case_id})
    sample_hit = bs.find_one({"external_references.id": "pgx:TCGA-" + sample_id})

    if case_hit is None:
        print("No matching case id found\n")

    else:
        print("Case id in database")
        n_hits = bs.count_documents({"external_references.id": "pgx:TCGA-" + case_id})
        print("Number of matches with case id = ", n_hits)
        #pprint.pprint(case_hit)
        print("\n")



    if sample_hit is None:
        print("No matching sample id found\n")

    else:
        print("Sample id already in database\n")
    print("#" * 79 + "\n")

In [ ]:
### Dictionary from dataframe --- usefull?
import pandas as pd
import pprint
from pymongo import MongoClient



df = pd.read_csv('../temp/intermediate_mapping_file.csv') ###### Make input later
df = df.loc[:, ~df.columns.isin(['all_effects', 'HGVSp'])]
newdata_dictionary = df.to_dict(orient="records")

# Prepare mongodb tools
client = MongoClient('localhost', 27017)
db = client.MyTCGA
bios_coll = db[ "biosamples" ]
#bios_coll.insert_many(newdata_dictionary)

for row, line in df.iterrows():
    sample_id = line[0]
    mns_id = line[1]
    case_id = line[2]
    info = line[3:]
    print(info)

In [ ]:
import pymongo
import pandas as pd

# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['mydatabase']
collection = db['mycollection']

# Create sample DataFrame
df = pd.DataFrame({
    'id': ['1', '2', '3', '4'],
    'name': ['Alice', 'Bob', 'Charlie', 'Dave'],
    'age': [25, 30, 35, 40]
})

# Loop through DataFrame and check for matches in MongoDB
for i, row in df.iterrows():
    match = collection.find_one({'id': row['id']})
    if match:
        print(f"Match found for ID {row['id']}: {match}")
    else:
        print(f"No match found for ID {row['id']}. Adding to database.")
        collection.insert_one({
            'id': row['id'],
            'name': row['name'],
            'age': row['age']
        })
